## Authenticating on Google Earth Engine

In [5]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code:  4/1AX4XfWjlcuEsbImN3gaZrMEOhABBjDMwabAgxOM-ojzWKLhu4TKI1JMtfbI



Successfully saved authorization token.


## Creating an image collection based on Gridded Population of the World Version 4.11

In [6]:
#https://developers.google.com/earth-engine/datasets/catalog/CIESIN_GPWv411_GPW_Population_Count#description
# 'l' is for visualization purposes
l = ee.ImageCollection("CIESIN/GPWv411/GPW_Population_Count").first()

## Basic visualization of population count on map

In [7]:
# 'e' is to visualize population count
e = l.select('population_count')

In [8]:
from IPython.display import Image

#visualising basic population count on map

# Create a URL to the styled image for a region around France.
url = e.getThumbUrl({
  "max": 1000.0,
  "palette": [
    "ffffe7",
    "86a192",
    "509791",
    "307296",
    "2c4484",
    "000066"
  ],"dimensions":4096,
  "min": 0.0
})
print(url)

# Display the thumbnail land surface temperature in France.
print('\nPlease wait while the thumbnail loads, it may take a moment...')
Image(url=url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/49068f3e80a22ac32782551349000840-2bf0aec8f06d2abc5edb740e064484ef:getPixels

Please wait while the thumbnail loads, it may take a moment...


## Creating dataframe with population count (2020) according to coordinates 

In [26]:
radius_ = 2
grid_size =int(f'{radius_}')

type(grid_size)

int

In [9]:
# 'd' is to create dataframe (the same as 'l' but excluding '.first()'
d= ee.ImageCollection("CIESIN/GPWv411/GPW_Population_Count")

In [10]:
#scale = ee.ImageCollection("CIESIN/GPWv411/GPW_Population_Count").first().projection().nominalScale

In [11]:
lat, lon = -31.95189238136466, 115.860672905644
#scale remains constant
scale = 1000
#u_poi = ee.Geometry.Point(lon, lat)
#grid size in km
grid_size = 100

In [12]:
from haversine import inverse_haversine, Direction
radius=(grid_size * 2**0.5 )/2

coord_ne = inverse_haversine((lat,lon), radius, Direction.NORTHEAST)
coord_sw = inverse_haversine((lat,lon), radius, Direction.SOUTHWEST)

print(coord_sw)
print(coord_ne)

(-32.40044199160678, 115.32810775799535)
(-31.501141633938587, 116.38804987986838)


In [13]:
u_poi = ee.Geometry.Rectangle([
                                 [coord_sw[1], coord_sw[0]],
                                 [coord_ne[1], coord_ne[0]]
                                ])


In [14]:
e_u_poi = d.select('population_count').getRegion(u_poi, scale).getInfo()

In [15]:
import pandas as pd
df = pd.DataFrame(e_u_poi)

In [16]:
df.columns = df.iloc[0]

In [17]:
df = df[1:]
df = df.sort_values('time')

In [18]:
df.reset_index(inplace=True, drop=True)
df

,id,longitude,latitude,time,population_count
0,gpw_v4_population_count_rev11_2000_30_sec,115.330208,-32.397741,946684800000,None
1,gpw_v4_population_count_rev11_2000_30_sec,115.923096,-32.056381,946684800000,338.580566
2,gpw_v4_population_count_rev11_2000_30_sec,115.914113,-32.056381,946684800000,904.87616
3,gpw_v4_population_count_rev11_2000_30_sec,115.90513,-32.056381,946684800000,997.921509
4,gpw_v4_population_count_rev11_2000_30_sec,115.635635,-31.571291,946684800000,4.418503
...,...,...,...,...,...
58995,gpw_v4_population_count_rev11_2020_30_sec,115.887163,-31.903667,1577836800000,1865.578613
58996,gpw_v4_population_count_rev11_2020_30_sec,115.87818,-31.903667,1577836800000,1820.080566
58997,gpw_v4_population_count_rev11_2020_30_sec,115.869197,-31.903667,1577836800000,1718.911133
58998,gpw_v4_population_count_rev11_2020_30_sec,115.932079,-31.903667,1577836800000,324.085022


In [20]:
df = df.loc[df['time'] == 1577836800000]
population_count= int(df['population_count'].sum())

1430580.7345349477

## Failed Wikipedia Location Based Population Scraping

In [194]:
import requests
from bs4 import BeautifulSoup

In [205]:
inputt='miami'
response = requests.get(
    url=f"https://en.wikipedia.org/wiki/{inputt}",
)

In [206]:
soup = BeautifulSoup(response.content, 'html.parser')

In [207]:
title = soup.findAll('tr', class_= 'mergedrow')
print(title)

[<tr class="mergedrow"><th class="infobox-label" scope="row"><a href="/wiki/U.S._state" title="U.S. state">State</a></th><td class="infobox-data"><a href="/wiki/Florida" title="Florida">Florida</a></td></tr>, <tr class="mergedrow"><th class="infobox-label" scope="row"><a href="/wiki/County_(United_States)" title="County (United States)">Constituent counties</a> (<a href="/wiki/List_of_counties_in_Florida" title="List of counties in Florida">County</a>)</th><td class="infobox-data"><a href="/wiki/Miami-Dade_County,_Florida" title="Miami-Dade County, Florida">Miami-Dade</a></td></tr>, <tr class="mergedrow"><th class="infobox-label" scope="row"><a href="/wiki/List_of_regions_of_the_United_States" title="List of regions of the United States">Region</a></th><td class="infobox-data"><a href="/wiki/South_Atlantic_states" title="South Atlantic states">South Atlantic</a></td></tr>, <tr class="mergedrow"><th class="infobox-label" scope="row"><a href="/wiki/Municipal_corporation" title="Municipal

In [208]:
a= []
for element in title:
    a.append(element.text)

In [209]:
type(title)

bs4.element.ResultSet

In [210]:
b=[]
for x in a: 
    if "Total" in x:
        b.append(x)

In [211]:
c = b[-1]

In [212]:
d=c.strip('\xa0•\xa0Total')

In [214]:
e=d.replace(' ', '')[:-3]
e

'442,241'